# FinanceInfo

In [1]:
!pip3 install --user pandas
!pip3 install --user html5lib
!pip3 install --user BeautifulSoup4
!pip3 install --user plotly
!pip3 install --user nbformat

In [59]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

import datetime
from mplfinance.original_flavor import candlestick2_ohlc
import matplotlib.pyplot as plt

In [3]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

def get_url(item_name, code_df): 
  code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
  url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
  print("요청 URL = {}".format(url)) 
  return url

def get_html_table_symbol(url):
  response = requests.get(url, headers=headers)
  html = bs(response.text, 'lxml')
  html_table = html.select('table')
  len(html_table)
  return str(html_table)

In [4]:
def get_report_time():
  return datetime.datetime.now().strftime("%Y%m%d")

In [5]:
def get_stochastic(df, n=15, m=5, t=3):
  df = pd.DataFrame(df)
  ndays_high = df.high.rolling(window=n, min_periods=1).max()
  ndays_low = df.low.rolling(window=n, min_periods=1).min()

  # Fast%K
  kdj_k = ((df.close - ndays_low)/(ndays_high-ndays_low))*100 #n=15
  # Fast%D
  kdj_d = kdj_k.ewm(span=m).mean() #m=5
  # Slow%D
  kdj_j = kdj_d.ewm(span=t).mean() #t=3
  
  df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
  return df

In [47]:
class FinanceInfo:
  def __init__(self, item_name, page:10):
    self.item_name = item_name
    self.page = page
  
    # 1: Get event_code from 상장법인목록.xls
    code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
    code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
    code_df = code_df[['회사명', '종목코드']]
    code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

    # 2. Get stock data from naver finance
    url = get_url(self.item_name, code_df)

    df = pd.DataFrame()
    for _page in range(1, self.page):
      pg_url = '{url}&page={page}'.format(url=url, page=_page)
      table = get_html_table_symbol(pg_url)
      df = df.append(pd.read_html(table, header=0)[0], ignore_index=True)
    df = df.dropna()

    # 3. Rename columns
    df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
    df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 
    df['date'] = pd.to_datetime(df['date']) 
    df = df.sort_values(by=['date'], ascending=True)

    self.df = df

  def whoami(self):
    return self.item_name

  def get_page(self):
    return self.page

  def get_df(self):
    return self.df

  def get_stochastic(self):
    df = get_stochastic(self.df)

    fig = plt.figure(figsize=(20,20))
    ax1 = fig.add_subplot(5,1,1)
    ax2 = fig.add_subplot(5,1,2)
    ax3 = fig.add_subplot(5,1,3)
    ax4 = fig.add_subplot(5,1,4)
    ax5 = fig.add_subplot(5,1,5)

    ax4.plot(df.date, df['kdj_j'], 'r',label='Slow%D')
    ax3.plot(df.date, df['kdj_d'], 'g',label='Slow%K')
    ax2.plot(df.date, df['kdj_k'], 'b',label='Fast%K')

    ax5.plot (df.date, df['kdj_d'], 'g',label='Fast%D (Slow%K)')
    ax5.plot(df.date, df['kdj_j'] , 'r',label='Slow%D')

    candlestick2_ohlc(ax1, df['open'],df['high'],df['low'], df['close'], width=0.5, colorup='r', colordown='b')

    ax4.legend()
    ax3.legend()
    ax2.legend()

    plt.axhline(20)
    plt.axhline(80)

    plt.savefig('../report/%s-%s.pdf'%(self.item_name,get_report_time()))
    plt.close(fig)
    #return fig

In [48]:
if __name__ == '__main__':
  item = FinanceInfo('KG ETS',page=10)
  item.get_stochastic()

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=151860


# Stock Signals
http://www.stocksignals.co.kr:8080/stockguest/0025.php

In [151]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

code_df = pd.read_html('http://www.stocksignals.co.kr:8080/stockguest/0025.php', header=0)[8]
print(code_df)

url = 'http://www.stocksignals.co.kr:8080/stockguest/0025.php'

webpage = requests.get(url)
soup = BeautifulSoup(webpage.content, "html.parser")

html_ = str(soup.findAll("td", {"colspan":"11"}))
df = pd.read_html(html_)[1]
headers = df.iloc[0]
headers2 = df.iloc[1]

print(headers)
print(headers2)

df = pd.DataFrame(df.values[2:], columns=headers)
df.columns = pd.MultiIndex.from_tuples(zip(headers2,df.columns))

pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)

Empty DataFrame
Columns: [■ 투신과 연기금,외국인 동시순매수 종목현황 ( 2021-08-12 현재 )  기간선택  당일  5일  10일  20일  정렬선택  수량순  일수순  (단위 : 원,주,시가총액-억원,%)  종목명  시가총액  현재가  최근 1일  전일가  등락률  최고가  최저가  투신  연기금  외국인  일수  한화시스템19,95218,10018,100+7.7418,10016,800220,788181,816617,0761일삼강엠앤티*6,75019,00019,000+9.2019,00017,400182,78744,637242,5401일우리금융지주011,20011,200+0.4511,25011,200131,60979,5144,3701일카카오654,689147,500147,500+2.43147,500144,00079,911124,91313,8621일두산중공업91,67921,70021,700+1.6421,70021,35054,034115,1391,669,5251일JB금융지주15,6017,9207,920+2.337,9207,740103,52338,47251,3111일덕산테코피아*5,89932,10032,100+15.0532,10027,900124,38916,469163,2391일HMM140,40240,65040,650+1.2540,65040,15024,251108,292115,3541일와이지엔터테인먼트*11,35561,60061,600+11.5961,60055,20082,81035,59018,8801일브리지텍*7055,9005,900+2.795,9005,740114,98330097,1911일미래에셋생명7,7624,3854,385+0.924,3854,34579,99015,76842,6771일디아이씨2,5786,6306,630+7.636,6306,16094,13757203,3771일쌍용C&E40,6618,0708,070+1.008,0707,99052,72937,00580,4761일심텍*10,14631,85031,850+3.5831,85030,

In [152]:
#df.style.hide_index()
df
#print(df.iloc[0:1])
#print(df)

종       시       현       전       등       최               투       연        외   일
             목       가       재       일       락       고       저       신       기        국   수
0   한화시스템       19952   18100   18100   +7.74   18100   16800   220788  181816  617076   1일
1   삼강엠앤티*      6750    19000   19000   +9.20   19000   17400   182787  44637   242540   1일
2   우리금융지주      0       11200   11200   +0.45   11250   11200   131609  79514   4370     1일
3   카카오         654689  147500  147500  +2.43   147500  144000  79911   124913  13862    1일
4   두산중공업       91679   21700   21700   +1.64   21700   21350   54034   115139  1669525  1일
5   JB금융지주      15601   7920    7920    +2.33   7920    7740    103523  38472   51311    1일
6   덕산테코피아*     5899    32100   32100   +15.05  32100   27900   124389  16469   163239   1일
7   HMM         140402  40650   40650   +1.25   40650   40150   24251   108292  115354   1일
8   와이지엔터테인먼트*  11355   61600   61600   +11.59  61600   55200   82810   35590   18880    1일
9   브리지텍*       705     5900    5900    +2.79   5900    5740    114983  300     97191    1일
10  미래에셋생명      7762    4385    4385    +0.92   4385    4345    79990   15768   42677    1일
11  디아이씨        2578    6630    6630    +7.63   6630    6160    94137   57      203377   1일
12  쌍용C&E       40661   8070    8070    +1.00   8070    7990    52729   37005   80476    1일
13  심텍*         10146   31850   31850   +3.58   31850   30750   9086    72687   68578    1일
14  아시아나항공      13841   18600   18600   +0.81   18600   18450   346     74786   66601    1일
15  한화솔루션       75746   39600   39600   +0.64   39600   39350   68140   3203    120454   1일
16  오스코텍*       10829   36200   36200   +4.93   36200   34500   56101   14775   166940   1일
17  씨아이에스*      6928    12050   12050   +6.17   12050   11350   28267   38713   24627    1일
18  엘앤에프*       36306   129300  129300  +5.04   129300  123100  37612   15931   171771   1일
19  삼성증권        42150   47200   47200   +1.18   47200   46650   34996   18167   141637   1일
20  현대일렉트릭      8832    24500   24500   +1.24   24500   24200   14993   30143   75062    1일
21  한미반도체       17756   35900   35900   +1.41   35900   35400   1186    42453   18616    1일
22  테스나*        8147    55200   55200   +2.03   55200   54100   34877   8285    9895     1일
23  기아          351045  86600   86600   +4.21   86600   83100   18237   22874   2002842  1일
24  한솔로지스틱스     1357    4940    4940    +1.42   5010    4940    197     35281   81640    1일
25  JYP Ent.*   16134   45450   45450   +4.24   45450   43600   9018    26198   74895    1일
26  신한지주        203282  39350   39350   +0.38   39350   39200   4237    30400   265817   1일
27  LIG넥스원      9977    45350   45350   +2.49   45350   44250   4345    29427   34924    1일
28  DB하이텍       28548   64300   64300   +2.72   64300   62600   4377    25998   199277   1일
29  한국타이어앤테크놀로  60699   49000   49000   +0.93   49000   48550   6520    23838   35155    1일
30  레이언스*       2812    16950   16950   +1.50   16950   16700   17032   12569   41683    1일
31  HDC현대산업개발   19673   29850   29850   +1.19   29850   29500   3542    24546   98150    1일
32  일진머티리얼즈     37765   81900   81900   +2.76   81900   79700   7110    19178   113701   1일
33  SK케미칼       34432   293000  293000  +3.58   303500  293000  5402    20656   22781    1일
34  솔루스첨단소재     0       69900   69900   +4.48   69900   66900   9830    16017   100300   1일
35  명신산업        14803   36300   36300   +2.11   36300   35550   20474   3622    50528    1일
36  디오*         8336    53700   53700   +5.50   53700   50900   14168   8882    49454    1일
37  셀트리온제약*     59022   161500  161500  +3.39   161500  156200  20391   1296    33535    1일
38  대우조선해양      33099   30850   30850   +1.65   30850   30350   444     20461   4422     1일
39  한섬          9963    40450   40450   +3.85   40450   38950   136     19165   5398     1일
40  센코*         1871    22900   22900   +7.01   22900   21400   17272   2000    10723    1일
41  SKC         62104   164000  164000  +2.50   164000  1